In [1]:
import numpy as np
import pandas as pd

In [2]:
dong = pd.read_excel("geojson_행정동코드.xlsx") ## 누락된 행정동 이름 비교 후 입력
dis = pd.read_excel("요양기관.xlsx")

In [3]:
dong

,시,구,동,행정동코드
0,서울특별시,종로구,사직동,11110530
1,서울특별시,종로구,삼청동,11110540
2,서울특별시,종로구,부암동,11110550
3,서울특별시,종로구,평창동,11110560
4,서울특별시,종로구,무악동,11110570
...,...,...,...,...
419,서울특별시,강동구,둔촌1동,11740690
420,서울특별시,강동구,둔촌2동,11740700
421,서울특별시,강동구,암사1동,11740570
422,서울특별시,강동구,천호2동,11740610


In [4]:
dis

,Unnamed: 0,구,행정동별요양기관정원
0,4,종로구,461
1,5,중구,260
2,6,용산구,385
3,7,성동구,691
4,8,광진구,1004
5,9,동대문구,1036
6,10,중랑구,1643
7,11,성북구,1028
8,12,강북구,1166
9,13,도봉구,1870


In [5]:
# dis "구" 기준으로 정렬
dis.sort_values(by="구",inplace=True)

In [6]:
# dis index 다시 설정
dis.reset_index(inplace=True)

In [7]:
# 필요없는 컬럼 삭제
dis.drop(["Unnamed: 0"], axis=1, inplace=True)

In [8]:
# 필요없는 컬럼 삭제
dong.drop(["시","행정동코드"], axis=1, inplace=True)

In [9]:
dong_no = dong.groupby("구").count()
dong_no.sort_values(by="구",inplace=True)
dong_no.reset_index(inplace=True)
dong_no

,구,동
0,강남구,22
1,강동구,18
2,강북구,13
3,강서구,20
4,관악구,21
5,광진구,15
6,구로구,15
7,금천구,10
8,노원구,19
9,도봉구,14


In [10]:
dis["동"] = dong_no["동"]

In [11]:
# 동별장기요양병원 = 정원 / 동
dis["행정동별요양기관정원"] = dis["행정동별요양기관정원"] / dis["동"]

In [12]:
dong.sort_values("구")

,구,동
362,강남구,대치1동
375,강남구,압구정동
376,강남구,청담동
377,강남구,대치2동
378,강남구,개포2동
...,...,...
105,중랑구,신내1동
109,중랑구,망우본동
108,중랑구,면목3.8동
101,중랑구,중화2동


In [13]:
dong.sort_values(["구","동"], inplace=True)

In [14]:
# 각 구의 개수를 알고있기 때문에 그 수만큼 동* "동별장기요양병원" 대입
# 예를들어 강남구가 5개이고 "동별장기요양병원" 이 29.318182 일 경우
# 10 * [29.318182] 를 하면 [29.318182, 29.318182, 29.318182, 29.318182, 29.318182, 29.318182] 가 된다.
# 이를 다더하면 2차원 배열이 된다. 매 for 문 마다 배열이 dong_val 배열에 들어가기 때문에
final = []
for i in range(len(dis)):
    final += (dis.iloc[i]["동"] * [dis.iloc[i]["행정동별요양기관정원"]])

In [15]:
# 위에서 인덱스를 조정했기 때문에 인덱스를 다시 설정. 설정하지 않을 경우 데이터를 추가할 시 이상하게 추가된다
dong.reset_index(inplace=True)
dong

,index,구,동
0,368,강남구,개포1동
1,378,강남구,개포2동
2,369,강남구,개포4동
3,358,강남구,논현1동
4,359,강남구,논현2동
...,...,...,...
419,99,중랑구,상봉2동
420,105,중랑구,신내1동
421,106,중랑구,신내2동
422,100,중랑구,중화1동


In [16]:
t = pd.Series(final)
dong["행정동별요양기관정원"] = pd.Series(t)

In [17]:
dong

,index,구,동,행정동별요양기관정원
0,368,강남구,개포1동,29.318182
1,378,강남구,개포2동,29.318182
2,369,강남구,개포4동,29.318182
3,358,강남구,논현1동,29.318182
4,359,강남구,논현2동,29.318182
...,...,...,...,...
419,99,중랑구,상봉2동,102.687500
420,105,중랑구,신내1동,102.687500
421,106,중랑구,신내2동,102.687500
422,100,중랑구,중화1동,102.687500


In [18]:
# 필요없는 컬럼지우기
dong.drop("index", axis=1, inplace=True)

In [19]:
nums = pd.read_excel("geojson_행정동코드.xlsx") ## 행정동 코드 추가

In [20]:
nums

,시,구,동,행정동코드
0,서울특별시,종로구,사직동,11110530
1,서울특별시,종로구,삼청동,11110540
2,서울특별시,종로구,부암동,11110550
3,서울특별시,종로구,평창동,11110560
4,서울특별시,종로구,무악동,11110570
...,...,...,...,...
419,서울특별시,강동구,둔촌1동,11740690
420,서울특별시,강동구,둔촌2동,11740700
421,서울특별시,강동구,암사1동,11740570
422,서울특별시,강동구,천호2동,11740610


In [21]:
# 필요없는 컬럼 삭제
nums.drop(["시"], axis=1, inplace=True)

In [22]:
all_df = pd.merge(left=dong, right=nums,how='left',
                  on=['구','동'], sort=False)

In [23]:
all_df

,구,동,행정동별요양기관정원,행정동코드
0,강남구,개포1동,29.318182,11680660
1,강남구,개포2동,29.318182,11680670
2,강남구,개포4동,29.318182,11680690
3,강남구,논현1동,29.318182,11680521
4,강남구,논현2동,29.318182,11680531
...,...,...,...,...
419,중랑구,상봉2동,102.687500,11260590
420,중랑구,신내1동,102.687500,11260680
421,중랑구,신내2동,102.687500,11260690
422,중랑구,중화1동,102.687500,11260600


In [24]:
all_df = all_df[['행정동코드', '구', '동', '행정동별요양기관정원']]

In [25]:
all_df

,행정동코드,구,동,행정동별요양기관정원
0,11680660,강남구,개포1동,29.318182
1,11680670,강남구,개포2동,29.318182
2,11680690,강남구,개포4동,29.318182
3,11680521,강남구,논현1동,29.318182
4,11680531,강남구,논현2동,29.318182
...,...,...,...,...
419,11260590,중랑구,상봉2동,102.687500
420,11260680,중랑구,신내1동,102.687500
421,11260690,중랑구,신내2동,102.687500
422,11260600,중랑구,중화1동,102.687500


In [26]:
# all_df.to_excel("행정동별로_나눈_요양기관.xlsx")

In [27]:
import sklearn
sklearn.__version__

import pandas as pd
import numpy as np
import folium

from sklearn.preprocessing import MinMaxScaler

In [28]:
scaler= MinMaxScaler()

all_df[["행정동별요양기관정원"]]= scaler.fit_transform(all_df[["행정동별요양기관정원"]])

all_df

,행정동코드,구,동,행정동별요양기관정원
0,11680660,강남구,개포1동,0.103106
1,11680670,강남구,개포2동,0.103106
2,11680690,강남구,개포4동,0.103106
3,11680521,강남구,논현1동,0.103106
4,11680531,강남구,논현2동,0.103106
...,...,...,...,...
419,11260590,중랑구,상봉2동,0.734305
420,11260680,중랑구,신내1동,0.734305
421,11260690,중랑구,신내2동,0.734305
422,11260600,중랑구,중화1동,0.734305


In [29]:
all_df.to_excel("요양기관_정규화.xlsx")

In [30]:
low = all_df.sort_values(by = ['행정동별요양기관정원', '행정동코드'], ascending = True).head(10)
#low

In [31]:
high = all_df.sort_values(by = ['행정동별요양기관정원', '행정동코드'], ascending = False).head(10)
#high

In [32]:
low = low[['행정동코드', '구', '동']]
low

,행정동코드,구,동
397,11140520,중구,소공동
407,11140540,중구,회현동
396,11140550,중구,명동
405,11140570,중구,필동
402,11140580,중구,장충동
393,11140590,중구,광희동
401,11140605,중구,을지로동
399,11140615,중구,신당동
394,11140625,중구,다산동
400,11140635,중구,약수동


In [33]:
high = high[['행정동코드', '구', '동']]
high

,행정동코드,구,동
157,11320710,도봉구,방학3동
156,11320700,도봉구,방학2동
155,11320690,도봉구,방학1동
161,11320681,도봉구,쌍문4동
160,11320680,도봉구,쌍문3동
159,11320670,도봉구,쌍문2동
158,11320660,도봉구,쌍문1동
154,11320522,도봉구,도봉2동
153,11320521,도봉구,도봉1동
166,11320515,도봉구,창5동
